In [ ]:
#import fwiVis.fwiVis as fv
import s3fs
s3 = s3fs.S3FileSystem(anon=False)
from math import cos, asin, sqrt
import re

import numpy as np
import geopandas as gpd
import pandas as pd
from matplotlib import pyplot as plt
import os
import rioxarray as rio
import xarray as xr
import rasterio
import glob
from shapely.errors import ShapelyDeprecationWarning
from shapely.geometry import Point
import warnings
import folium
import datetime
import time
from folium import plugins
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning) 
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning) 
import contextily as cx
from shapely.geometry import box
import sys
from datetime import datetime, timedelta
from itertools import chain

from datetime import date


sys.path.insert(0, '/projects/old_shared/fire_weather_vis/base-fwi-vis/')
import fwiVis.fwiVis as fv

In [ ]:
### Functions that take largefire perimeters that are different overlapping fire and merge them into one. 
def prep_fire_files(path, crs = "3571"):
    '''
    Read in cvs of previously output GeoDataFrame as a GeoDataFrame with valid geometry column. 
    '''
    fires = pd.read_csv(path)
    fires = fires.rename(columns={"geometry":"csv_geometry"})
    fires.t = fires.t.astype("str")
    fires.fireID  = fires.fireID.astype("str")
    #fires['csv_geometry'] =fires['csv_geometry'].apply(wkt.loads)
    fires_geom = gpd.read_file(path, GEOM_POSSIBLE_NAMES="geometry", KEEP_GEOM_COLUMNS="NO")
    fires_geom.t = fires_geom.t.astype("str")
    fires_geom.fireID  = fires_geom.fireID.astype("str")
    fires = fires_geom[["fireID", "t", "geometry"]].merge(fires, on=["fireID", "t"], how = "left")
    fires = fires.set_crs(crs)
    return (fires)

def remerge_largefire(fires):
    '''
    If two final feds perimeters intersect spatialy, check if one ended before the other began. If yes, give it the ID of the earlier perimeter. 
    Note: Could optionally be spitting out time differences, or sorting by them. For Quebec, was a max of 22 days, min of 1 day. Theoretically, not sure why the 1 day perimeter wasn't merged by feds. 
    '''
    # Get an id, first/ last t per ID, and a geometry
    first_perims = fires[~fires.geometry.isnull()].groupby("fireID").t.min().reset_index()
    last_perims = fires[~fires.geometry.isnull()].groupby("fireID").t.max().reset_index()
    plot_last = fires.merge(last_perims, on = ["fireID", "t"], how = 'right')
    plot_first = fires.merge(first_perims, on = ["fireID", "t"], how = 'right')
    plot_last = plot_last[["fireID", "t", "geometry"]]
    plot_first = plot_first[["fireID", "t", "geometry"]]
                    
    # Check what perimeters spatially intersect into anouther one through time
    last_in_last = plot_last.sjoin(plot_last, how = "left", predicate = "intersects")
    lil = last_in_last.groupby(["fireID_left",]).t_right.min().reset_index()
    id_with_max_time = lil.merge(last_in_last[["t_right", "t_left", "geometry", "fireID_right", "fireID_left"]], on = ["t_right", "fireID_left"], how = "right")

    id_with_max_time = id_with_max_time.rename(columns={"fireID_right": "mergeID", 
                                     "fireID_left" :"fireID",
                                     "t_right":"mergeID_t",
                                     "t_left":"fireID_end_t"
    })

    id_with_max_time_check = id_with_max_time[id_with_max_time.fireID_end_t < id_with_max_time.mergeID_t]

    ### FireID the earlier perimeter that later perimeters are merged into. "mergeID" describes the merge-ey ## I actually think this is wrong. I think I made it so the "fireID" is what is merged into the "mergeID"
    fireID_with_merge = id_with_max_time_check.groupby(["fireID"]).mergeID.unique().reset_index() 
    #print(fireID_with_merge)

    mergeID_with_fireID  = id_with_max_time_check.groupby(["mergeID"]).fireID.unique().reset_index()
    #print(mergeID_with_fireID)
                    
    # Check when the fireID and mergeID started/stopped. 
    get_merge_start = plot_first[["fireID", "t"]].rename(columns={"fireID":"mergeID", 
                                                           "t":"mergeID_start_t"})
    #print(get_merge_start)
    get_fireID_start =  plot_first[["fireID", "t"]].rename(columns={ "t":"fireID_start_t"})
    #print(get_fireID_start)

    id_map = id_with_max_time_check.merge(get_merge_start, on = ["mergeID"])
    id_map = id_map.merge(get_fireID_start, on = ["fireID"])
    id_map = id_map[["fireID","fireID_start_t",  "fireID_end_t", "mergeID", "mergeID_start_t", "mergeID_t"]]
    id_map["time_diff_fireIDend_mergeIDstart"] = id_map.fireID_end_t.astype('datetime64[ns]') - id_map.mergeID_start_t.astype('datetime64[ns]') ## Negative means that mergeID started after fireID ended
    #print(id_map)
    # Below code is just for isolateing perimeters that ended before the later perimetetr began. This is useful for identifying places where the assumptino to invalidate a fire after five days might be wrong. 
#     # Subset to IDs where one fire "ended" before the next fire began                
#     only_IDs_with_negative_dates = id_map[id_map.time_diff_fireIDend_mergeIDstart.dt.days < 0]
    
#     # Go through an reindex just the IDs that overlap in space but not time                
#     fires["old_id"] = ""
#     fire_ids = only_IDs_with_negative_dates.mergeID.unique() ## Gives the IDs of fires to be merged into anouther fire

#     for i in fire_ids:
#         print("start: ")
#         print(fire_ids)
#         min_t = id_with_max_time_check[id_with_max_time_check.mergeID == i].fireID_end_t.min()
#         print(min_t)
#         sm_id_map = id_with_max_time_check[(id_with_max_time_check.mergeID == i) & (id_with_max_time_check.fireID_end_t == min_t)]
#         print(sm_id_map.fireID.values)
#         if(len(sm_id_map) == 0):
#             print("ID", i, " only merges with self")
#             fires["old_id"][fires.fireID == i] = i
#         else:
#             if(len(sm_id_map) != 1):
#                 print("There are two perimeters that intersect with ID",i, " that started at the same time.")
#                 print(sm_id_map)
#                 break

#             fires["old_id"][fires.fireID == i] = i
#             fires["fireID"][fires.fireID == i] = str(*sm_id_map.fireID.values)
                    
#     #return(fires)
    return(id_map)

In [ ]:
#path = "/projects/old_shared/fire_weather_vis/Lightning_analysis/fwi_timeline_merge/20_days_fire_stats_only_718270-99999_minT20230301_maxT20230830_BOREAL_NRT_3571_DPSstation20231120.csv" ## looking for lightning 20 days before start
path = "/projects/old_shared/fire_weather_vis/Lightning_analysis/fwi_timeline_merge/Final_dataset_as_of_20240209.csv" 
fires = prep_fire_files(path)

In [ ]:
fires = fires[fires.InterCloud.isna()]
fires.t = pd.to_datetime(fires.t, format='ISO8601')

In [ ]:
new_fires = remerge_largefire(fires)

In [ ]:
print(len(fires.fireID.unique()))

print(len(new_fires.fireID.unique()))

In [ ]:
### Me trying overlay: Doesn't seem to work. 

fires_exp = fires[fires.fireID.isin(["10406", "8495"])]

##
tmp_fires = remerge_largefire(fires_exp)

for i in tmp_fires.fireID:
    id_map = tmp_fires[tmp_fires.fireID == i]
    small_fr = fires_exp[fires_exp.fireID == i]
    small_fr.t = small_fr.t.astype("datetime64[ns]")
    small_fr["date"] = small_fr.t
   
    
    #small_fr = small_fr.set_index("t")
    #small_fr = small_fr.dropna()
    
    
    
    big_fire = fires_exp[fires_exp.fireID == id_map.mergeID.values[0]]
    big_fire.t = big_fire.t.astype("datetime64[ns]")
    big_fire["date"] = big_fire.t
   
    #big_fire = big_fire.set_index("t")
    
    ## They need the same index to do the overlay. Gotta merge them both into the same index
    
    min_t = min(small_fr.t.min(), big_fire.t.min())
    max_t = max(small_fr.t.max(), big_fire.t.max())
    
    times = pd.date_range(start = min_t, end = max_t, freq = "12H")
    
    times = pd.DataFrame({"t": times})
    
    small_fr = small_fr.merge(times, on = ["t"], how = "outer")
    small_fr = small_fr.sort_values(by = ["t"])
    small_fr = small_fr.reset_index()
    big_fire = big_fire.merge(times, on = ["t"], how = "outer")
    big_fire = big_fire.sort_values(by = ["t"])
    big_fire = big_fire.reset_index()
    big_fire = big_fire[["t", "geometry", "date"]]
    small_fr = small_fr[["t", "geometry", "date"]]
    print("Big fire min:",  big_fire.t.min())
    print("small fire min:",  small_fr.t.min())
    
    
    both_fires = gpd.overlay(small_fr, big_fire, how = "union", keep_geom_type = False)
    
    
#fires_exp["former_fire_IDS"] = fires_exp

In [ ]:
### Correcting or fires that merge into more than one fire
def ultimate_merge(new_fires):
    
    for i in new_fires.fireID.unique(): 
        print(i)
        possible_merge_ids = new_fires[new_fires.fireID == i].mergeID# get the fireIDs assosiated with that
        term_pair = []
        for m in possible_merge_ids:  
            print("Possible Merge IDS", possible_merge_ids)
            if not np.any(new_fires.fireID.astype("str").isin([str(m)])): #this implies is a terminator
                new_fires.loc[new_fires.fireID.astype("str") == str(i), "mergeID"] = m
                print("M is", m)
                term_pair = list(set(term_pair))
        if(len(term_pair) > 1):
            print(i, "has more than one terminator ID!")
            print(term_pair)
            next
    new_fires = new_fires[["fireID", "mergeID"]].drop_duplicates()
    return(new_fires)
        
def assign_last_perimeter(df, col_name = "is_last_perim"):
    if(np.any(df.columns.isin([col_name]))):
        max_t = df[~df.geometry.isna()].t.max()
        df.loc[df.t == max_t, col_name] = True
        return(df)
    else:
        print("Missing column" + " "+ col_name)
        ValueError()
    
    
                    
                    
                
    

In [ ]:
test_new_fires = ultimate_merge(new_fires)

In [ ]:
#remerge_largefire(fire_exp)
test_new_fires[test_new_fires.mergeID == "8495"]

In [ ]:
## Find terminator ID

#def find_term_id(df):
    

In [ ]:
not np.any([False, False, False])

In [ ]:
## Trying dissolve

#diss = fires_exp.dissolve(by = ["t"]) ## This does work.

tmp_fires = remerge_largefire(fires_exp)

for i in tmp_fires.fireID: # For each fire that will be merged into another fire
    id_map = tmp_fires[tmp_fires.fireID == i]
    small_fr = fires_exp[fires_exp.fireID == i]
    small_fr.t = small_fr.t.astype("datetime64[ns]")
    small_fr["date"] = small_fr.t
   
    
    #small_fr = small_fr.set_index("t")
    #small_fr = small_fr.dropna()
    
    
    
    big_fire = fires_exp[fires_exp.fireID == id_map.mergeID.values[0]]
    big_fire.t = big_fire.t.astype("datetime64[ns]")
    big_fire["date"] = big_fire.t

In [ ]:
test_if_double_merges = new_fires.groupby(["fireID", "mergeID"]).count().reset_index()
print(len(test_if_double_merges.fireID), len(test_if_double_merges.fireID.unique()))

In [ ]:
test_if_double_merges[test_if_double_merges.fireID.duplicated()]

In [ ]:
test_if_double_merges[test_if_double_merges.fireID == "8746"]

In [ ]:
new_fires[new_fires.fireID.isin(new_fires.mergeID)].fireID ## Intermediate fireIDs

new_fires[new_fires.fireID.isin(new_fires.mergeID)].mergeID ## The IDS the intermediate fireIDS will become

#sudo code: subset to intermediate ids, reassign mergeIDs to be ultimate ID
    # then do a while loop to continue the process untill the IDs in the "fireID" don't match any in the "mergeID". 


In [ ]:
tmp_fires

In [ ]:
#diss.index = diss.index.astype("str")

In [ ]:
#diss[(diss.index == "2023-06-23 12:00:00")][["geometry", "fireID"]].explore(style_kwds = {"fillOpacity": 0})

In [ ]:
#diss[["geometry", "fireID"]].dropna()#.explore(style_kwds = {"fillOpacity": 0})

In [ ]:
fires[(fires.fireID == "16062") & (~fires.geometry.isnull())][["fireID", "t"]].sort_values(by = "t")

In [ ]:
# Pull out each fire before it merges with another fire. Treat the fires post-merge as unique, with a composite ID

def keep_fires_seperate(newfire, fires):
    '''
    newfire [DataFrame] output from remerge_largefire. 
    fires [GeoDataFrame] geodataframe of all largefires. 
    '''
    fires.t = fires.t.astype("datetime64[ns]")
    fires["modified_id"] = fires.fireID
    newfire = newfire.sort_values(by = ["fireID", "fireID_start_t"])
    newfire.mergeID_start_t = newfire.mergeID_start_t.astype("datetime64[ns]")
    newfire.mergeID_t = newfire.mergeID_t.astype("datetime64[ns]")
    for m in newfire.mergeID.unique(): ### For each id that had something merged into it
        fireIDs = newfire[newfire.mergeID == m].fireID.unique() ## Get all the fireID of things that were merged into it
        for f in fireIDs: ## For each of those things that were merged into to 
            f = str(f)
            #print(m)
            #print(newfire[(newfire.fireID == i) & (newfire.mergeID == m)].mergeID_start_t.min())
            row_mask = (fires.fireID == m) & (fires.t >= newfire[(newfire.fireID == f) & (newfire.mergeID == m)].fireID_end_t.min()) & (fires.t <= newfire[(newfire.fireID == f) & (newfire.mergeID == m)].mergeID_t.max())
            #print(fires.loc[row_mask, ["modified_id"]])
            fires.loc[row_mask, ["modified_id"]] = fires.loc[row_mask, ["modified_id"]] + "_" + str(f) #fires.loc[(fires.fireID == m) & (fires.t >= newfire[(newfire.fireID == i) & (newfire.mergeID == i)].mergeID_start_t.min()) & (fires.t <= newfire[(newfire.fireID == i) & (newfire.mergeID == i)].mergeID_t.max()), ["modified_id"]] + "_" + str(i)
    return(fires)

In [ ]:
new_fires[new_fires.mergeID == '8495']

#new_fires[(new_fires.fireID == f) & (new_fires.mergeID == m)].fireID_end_t.min()

In [ ]:
fires_cp = fires
test_keep = keep_fires_seperate(new_fires, fires_cp)

In [ ]:
#fires_cp.loc[row_mask, ["modified_id"]] = "derp"

In [ ]:
test_keep[["fireID", "t", "modified_id"]]

In [ ]:
test_keep.loc[(test_keep.modified_id.str.contains("8495_")),["fireID","t", "modified_id"]]


In [ ]:
test_keep.loc[(test_keep.modified_id.str.contains("8505")),["fireID", "modified_id", "geometry"]].explore(style_kwds = {"fillOpacity": 0}, column = "modified_id")

In [ ]:
find_multi_merges = new_fires.groupby("fireID").count()
find_multi_merges[find_multi_merges.mergeID > 1]

In [ ]:
test_keep.loc[(test_keep.modified_id.str.contains("8495")),["fireID", "modified_id", "geometry"]]

In [ ]:
### Get the centriods of each subfire. 
test_keep["is_last_perim"] = False
last_of_unmerged_fires = test_keep.groupby("modified_id").apply(assign_last_perimeter)

In [ ]:
last_of_unmerged_fires = last_of_unmerged_fires[last_of_unmerged_fires.is_last_perim == True]
last_of_unmerged_fires

In [ ]:
# gdf_test['lon'] = gdf_test.centroid.x
# gdf_test['lat'] = gdf_test.centroid.y

last_of_unmerged_fires = last_of_unmerged_fires.to_crs("EPSG:4326")

last_of_unmerged_fires['lon'] = last_of_unmerged_fires.centroid.x
last_of_unmerged_fires['lat'] = last_of_unmerged_fires.centroid.y



last_of_unmerged_fires['fireID'] = last_of_unmerged_fires['modified_id']
last_of_unmerged_fires['data_source'] =  "/projects/old_shared/fire_weather_vis/Lightning_analysis/fwi_timeline_merge/Final_dataset_as_of_20240209.csv" 


last_of_unmerged_fires = last_of_unmerged_fires[['lon', 'lat',
       'farea','data_source', 'fireID']]

In [ ]:
last_of_unmerged_fires.to_csv("/projects/old_shared/fire_weather_vis/Lightning_analysis/lf_centroids/" + "March27_centroids_from_lf_kept_umerged_merged_seperate.csv")

#/projects/old_shared/fire_weather_vis/Lightning_analysis/lf_centroids/

In [ ]:
last_of_unmerged_fires.columns

In [ ]:
# fires_exp.t = fires_exp.t.astype("str")
#fires_exp[['fireID', 't', 'geometry']][fires_exp.t == "2023-06-23 12:00:00"].explore(style_kwds = {"fillOpacity": 0})

In [ ]:
bbox = fires[(fires.fireID == "8495") & (~fires.geometry.isna()) & (fires.t == fires[(fires.fireID == "8495") & (~fires.geometry.isna())].t.max())].bounds
fires.t = fires.t.astype("datetime64[ns]")
fires = fires.sort_values(by = "t", ascending= False)
fires.t = fires.t.astype("str")
fires[['fireID', 't', 'geometry']].cx[bbox.values[0][0]:bbox.values[0][2], bbox.values[0][1]:bbox.values[0][3]].explore(style_kwds = {"fillOpacity": 0}, column = "fireID")

In [ ]:
# some_ids = fires[['fireID', 't', 'geometry']].cx[bbox.values[0][0]:bbox.values[0][2], bbox.values[0][1]:bbox.values[0][3]].fireID.unique()

# new_fires[new_fires.fireID.isin(some_ids) | new_fires.mergeID.isin(some_ids)]

In [ ]:
#fires[['fireID', 't', 'geometry']][fires.t == "2023-06-01 12:00:00"].cx[bbox.values[0][0]:bbox.values[0][2], bbox.values[0][1]:bbox.values[0][3]].explore(style_kwds = {"fillOpacity": 0}, column = "fireID")

In [ ]:
#bbox.values[0]

In [ ]:
#sub_new = new_fires[new_fires.fireID.isin(some_ids) | new_fires.mergeID.isin(some_ids)]

In [ ]:
#ultimate_merge(sub_new)

In [ ]:
#sub_new

In [ ]:
# # 8645
# foi = '8645'

# bbox = fires[(fires.fireID == foi) & (~fires.geometry.isna()) & (fires.t == fires[(fires.fireID == foi) & (~fires.geometry.isna())].t.max())].bounds
# fires.t = fires.t.astype("datetime64[ns]")
# fires = fires.sort_values(by = "t", ascending= False)
# fires.t = fires.t.astype("str")

# some_ids = fires[['fireID', 't', 'geometry']].cx[bbox.values[0][0]:bbox.values[0][2], bbox.values[0][1]:bbox.values[0][3]].fireID.unique()
# fires[['fireID', 't', 'geometry']].cx[bbox.values[0][0]:bbox.values[0][2], bbox.values[0][1]:bbox.values[0][3]].explore(style_kwds = {"fillOpacity": 0}, column = "fireID")


In [ ]:
#new_fires[new_fires.fireID.isin(some_ids) | new_fires.mergeID.isin(some_ids)]

In [ ]:
# colors = {'10887':'red', '10978':'pink', '10718':'blue', '8645':'purple'}
# tmp_fires.fireID.map(colors)

In [ ]:
#plt.scatter(np.log(fire3.farea_diff), fire3.FWI, c = fire3.field_response_type.map(colors))

# tmp_fires = fires[['fireID', 't', 'geometry']][fires.fireID.isin(some_ids)]
# tmp_fires = tmp_fires.dropna()
# tmp_fires["color"] = tmp_fires.fireID.map(colors)

# tmp_fires.t = tmp_fires.t.astype("datetime64[ns]")
# tmp_fires = tmp_fires.sort_values(by = ["t"])
# tmp_fires.t = tmp_fires.t.astype("str")
# for t in tmp_fires.t:
#     print(t)
#     foo = tmp_fires.copy()
#     foo[foo.t != t] = None
#     foo.plot(facecolor = "none", column = tmp_fires.fireID)
#     plt.show()
    

In [ ]:
# fires[['fireID', 't', 'geometry']].cx[bbox.values[0][0]:bbox.values[0][2], bbox.values[0][1]:bbox.values[0][3]].plot( facecolor = "none", column = "fireID")

In [ ]:
#	8244


# foi = '8244'

# bbox = fires[(fires.fireID == foi) & (~fires.geometry.isna()) & (fires.t == fires[(fires.fireID == foi) & (~fires.geometry.isna())].t.max())].bounds
# fires.t = fires.t.astype("datetime64[ns]")
# fires = fires.sort_values(by = "t", ascending= False)
# fires.t = fires.t.astype("str")

# some_ids = fires[['fireID', 't', 'geometry']].cx[bbox.values[0][0]:bbox.values[0][2], bbox.values[0][1]:bbox.values[0][3]].fireID.unique()
# fires[['fireID', 't', 'geometry']].cx[bbox.values[0][0]:bbox.values[0][2], bbox.values[0][1]:bbox.values[0][3]].explore(style_kwds = {"fillOpacity": 0}, column = "fireID")

In [ ]:
### Need to fogure out if I can find more of the merge history in the allfire object, because I'm breaking my mind trying to recreate it
## Um actually, the fireIDs in the snapshots and the serialization files don't seem to match. This could be something else going on with the file generation, so I wont panic, but seems pretty weird. 

# def raw_pixel_times(fireID, date_string, year = "2023", path_region = "QuebecGlobalNRT_3571"):
#     '''
#     Function that finds raw times assosiated with VIIRS observations given a datetime and a fireID. 
#     Will open up serialization file pickle object to find data. 
#     Recommended to make the date-string as late in fire timeseries as possbile. 
        
#     INPUTS:
        
#         fireID (int): fireID of fire of interest.
#         date_string (str): Date of serialization file to read in. Format of %Y%m%d%p  ex: '20230801AM'.
#         year (str): Year that fires took place. Default to 2023. Availible options differ by path_region. 
#         path_region (str): This constructs the path that the fires are stored in. Default to QuebecGlobalNRT_3571/ 
        
#     '''
#     import pickle
#     import pandas as pd
#     import os
#     import sys
    
#     sys.path.insert(0, '/projects/fireatlas_nrt/')
#     path = "/projects/shared-buckets/gsfc_landslides/FEDSoutput-s3-conus/"+ path_region + "/"+ year + "/Serialization/" + date_string + ".pkl"

#     # open a file, where you stored the pickled data
#     file = open(path, 'rb')

#     # dump information to that file
#     data = pickle.load(file)

#     # close the file
#     file.close()
    
#     fireID = int(fireID)
    
#     times = []
#     for i in range(0, len(data.fires[fireID].pixels)):
#         #print(i)
#         times.append(data.fires[fireID].pixels[i].datetime)
    
#     times = pd.DataFrame(times, columns = ['t'])
#     times = times.sort_values(by = ['t'])
#     times["count"] = 1
#     times_grp = times.groupby("t").count()
#     times_grp["fireID"] = str(fireID)
    
#     return(times_grp)

In [ ]:
# import pickle
# import pandas as pd
# import os
# import sys


# fireID = 8244
# date_string = "20230601PM"
# year = "2023"
# path_region = "QuebecGlobalNRT_3571"
# sys.path.insert(0, '/projects/fireatlas_nrt/')
# path = "/projects/shared-buckets/gsfc_landslides/FEDSoutput-s3-conus/"+ path_region + "/"+ year + "/Serialization/" + date_string + ".pkl"

# snapshot = fv.prep_gdf(date = date_string,layer='perimeter', handle_multi=False,only_lf=True,area_lim=5, year = "2023", crs = fires.crs, path_region = path_region)

# file = open(path, 'rb')

# # dump information to that file
# data = pickle.load(file)

# # close the file
# file.close()


In [ ]:
# snapshot.cx[bbox.values[0][0]:bbox.values[0][2], bbox.values[0][1]:bbox.values[0][3]].explore()

In [ ]:
#data.fires[8244]

In [ ]:
#data.fires

In [ ]:
#fires.columns.isin(["fireID"])

In [ ]:
fires["is_last_perim"] = False

last_only = fires.groupby("fireID").apply(assign_last_perimeter)

In [ ]:
last_only = last_only[last_only.is_last_perim == True]

In [ ]:
last_only = last_only[~last_only.geometry.isna()]

last_only.t = last_only.t.astype("str")
last_only[['fireID', 't', 'geometry']].explore(style_kwds = {"fillOpacity": 0}, column = "fireID")

In [ ]:

foi = '12596'

bbox = fires[(fires.fireID == foi) & (~fires.geometry.isna()) & (fires.t == fires[(fires.fireID == foi) & (~fires.geometry.isna())].t.max())].bounds
fires.t = fires.t.astype("datetime64[ns]")
fires = fires.sort_values(by = "t", ascending= False)
fires.t = fires.t.astype("str")

some_ids = fires[['fireID', 't', 'geometry']].cx[bbox.values[0][0]:bbox.values[0][2], bbox.values[0][1]:bbox.values[0][3]].fireID.unique()
fires[['fireID', 't', 'geometry']].cx[bbox.values[0][0]:bbox.values[0][2], bbox.values[0][1]:bbox.values[0][3]].explore(style_kwds = {"fillOpacity": 0}, column = "fireID")

In [ ]:
#new_fires[new_fires.fireID.isin(some_ids) | new_fires.mergeID.isin(some_ids)][['fireID', 'mergeID', 'fireID_start_t', 'fireID_end_t']].drop_duplicates()

In [ ]:
#new_fires[new_fires.fireID == "12603"]

In [ ]:
# sample = ultimate_merge(new_fires[new_fires.fireID.isin(some_ids) | new_fires.mergeID.isin(some_ids)][['fireID', 'fireID_start_t', 'fireID_end_t', 'mergeID',
#        'mergeID_start_t']].drop_duplicates())
# sample[['fireID', 'mergeID','fireID_start_t', 'fireID_end_t']].drop_duplicates()

In [ ]:
# fires[['fireID', 't', 'geometry']][(fires.fireID == "12596") & (~fires.geometry.isna())].explore(style_kwds = {"fillOpacity": 0}, column = "t")

In [ ]:
# fires[['fireID', 't', 'geometry']][(fires.fireID == "12595") & (~fires.geometry.isna())].explore(style_kwds = {"fillOpacity": 0}, column = "t")

In [ ]:
# #new_fires.co

# sample.columns

In [ ]:
# print(fires[['fireID', 't', 'geometry']][(fires.fireID == "12595") & (~fires.geometry.isna())].t.min())
# print(fires[['fireID', 't', 'geometry']][(fires.fireID == "12595") & (~fires.geometry.isna())].t.max())

In [ ]:
# print(fires[['fireID', 't', 'geometry']][(fires.fireID == "12596") & (~fires.geometry.isna())].t.min())
# print(fires[['fireID', 't', 'geometry']][(fires.fireID == "12596") & (~fires.geometry.isna())].t.max())

In [ ]:
# fires[['fireID', 't', 'geometry']][(fires.fireID == "12595") & (~fires.geometry.isna()) & (fires.t == "2023-07-14 00:00:00" )].explore(style_kwds = {"fillOpacity": 0})

In [ ]:
# fires = fires[fires.fireID.isin(["12595", "12596", "12603"])]
# fires

In [ ]:
# first_perims = fires[~fires.geometry.isnull()].groupby("fireID").t.min().reset_index()
# last_perims = fires[~fires.geometry.isnull()].groupby("fireID").t.max().reset_index()
# plot_last = fires.merge(last_perims, on = ["fireID", "t"], how = 'right')
# plot_first = fires.merge(first_perims, on = ["fireID", "t"], how = 'right')
# plot_last = plot_last[["fireID", "t", "geometry"]]
# plot_first = plot_first[["fireID", "t", "geometry"]]

# plot_last[["fireID", "geometry"]].explore()

In [ ]:
# # Check what perimeters spatially intersect into anouther one through time
# last_in_last = plot_last.sjoin(plot_last, how = "left", predicate = "intersects")
# last_in_last

In [ ]:
# lil = last_in_last.groupby(["fireID_left",]).t_right.min().reset_index() ### Should this be t_left??? # Saying "for each fireID, give me the earlyiest time something that intersected with it ended"
# lil

In [ ]:
# id_with_max_time = lil.merge(last_in_last[["t_right", "t_left", "geometry", "fireID_right", "fireID_left"]], on = ["t_right", "fireID_left"], how = "right")
# id_with_max_time ## Give me the earliest merging. Note: I think this is why some of the merge combinations are missing? Only selects earlies one, but if multiple skips over them? 

In [ ]:
# id_with_max_time = id_with_max_time.rename(columns={"fireID_right": "mergeID", 
#                                  "fireID_left" :"fireID",
#                                  "t_right":"mergeID_t",
#                                  "t_left":"fireID_end_t"
# })
# id_with_max_time


In [ ]:
# id_with_max_time_check = id_with_max_time[id_with_max_time.fireID_end_t < id_with_max_time.mergeID_t]
# id_with_max_time_check ## Well, this actually looks correct. Although, I don't understand why there are two 12603 values? Oh, it merges first into 12596 then into 12595

In [ ]:
# ### FireID the earlier perimeter that later perimeters are merged into. "mergeID" describes the merge-ey ## I actually think this is wrong. I think I made it so the "fireID" is what is merged into the "mergeID"
# ## Aka "FireID" is merged into "MergeID"
# fireID_with_merge = id_with_max_time_check.groupby(["fireID"]).mergeID.unique().reset_index() 
# fireID_with_merge
# #print(fireID_with_merge)

In [ ]:
# mergeID_with_fireID  = id_with_max_time_check.groupby(["mergeID"]).fireID.unique().reset_index()
# mergeID_with_fireID
# #print(mergeID_with_fireID)

In [ ]:
# # Check when the fireID and mergeID started/stopped. 
# get_merge_start = plot_first[["fireID", "t"]].rename(columns={"fireID":"mergeID", 
#                                                        "t":"mergeID_start_t"})
# get_merge_start ## Correct? 

In [ ]:
# #print(get_merge_start)
# get_fireID_start =  plot_first[["fireID", "t"]].rename(columns={ "t":"fireID_start_t"}) ### UMMM mistake? 
# #print(get_fireID_start)

# get_fireID_start

In [ ]:
# id_map = id_with_max_time_check.merge(get_merge_start, on = ["mergeID"])
# id_map

In [ ]:
# id_map = id_map.merge(get_fireID_start, on = ["fireID"])
# id_map

In [ ]:
# id_map = id_map[["fireID","fireID_start_t",  "fireID_end_t", "mergeID", "mergeID_start_t", "mergeID_t"]]
# id_map

In [ ]:
# id_map["time_diff_fireIDend_mergeIDstart"] = id_map.fireID_end_t.astype('datetime64[ns]') - id_map.mergeID_start_t.astype('datetime64[ns]') ## Negative means that mergeID started after fireID ended
# id_map

In [ ]:
# test_ult = ultimate_merge(id_map)
# test_ult

In [ ]:
# print(fires[['fireID', 't']][(fires.fireID == "12603") & (~fires.geometry.isnull())].t.min())
# print(fires[['fireID', 't']][(fires.fireID == "12603") & (~fires.geometry.isnull())].t.max())